In [6]:
bank_to_package = {
    "CBE": "com.cbe.cbe",                      # Matches 'CBE' in your CSV
    "BOA": "com.boa.mobilebanking",            # Matches 'BOA'
    "Dashen": "com.dashen.mobilebanking"       # Matches 'Dashen'
}

In [2]:
import mysql.connector
from mysql.connector import Error

def create_connection():
    connection = None
    try:
        connection = mysql.connector.connect(
            host='localhost',  # e.g., 'localhost'
            user='root',  # your MySQL username
            password='mysql260394',  # your MySQL password
            database='bank_reviews'  # your database name
        )
        if connection.is_connected():
            print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

# Example usage
conn = create_connection()

Connection to MySQL DB successful


In [9]:
# Add sentiment labels based on sentiment_score
df['sentiment'] = df['sentiment_score'].apply(
    lambda x: 'POSITIVE' if x > 0.05 else 'NEGATIVE' if x < -0.05 else 'NEUTRAL'
)

In [10]:
import mysql.connector
import pandas as pd

# Load CSV and add sentiment labels
df = pd.read_csv('sentiment_themes_results.csv')
df['sentiment'] = df['sentiment_score'].apply(
    lambda x: 'POSITIVE' if x > 0.05 else 'NEGATIVE' if x < -0.05 else 'NEUTRAL'
)

# Bank name to package mapping
bank_to_package = {
    "CBE": "com.cbe.cbe",
    "BOA": "com.boa.mobilebanking",
    "Dashen": "com.dashen.mobilebanking"
}

# MySQL Connection
try:
    connection = mysql.connector.connect(
        host="localhost",
        user="bank_user",
        password="mysql260394",
        database="bank_reviews"
    )
    cursor = connection.cursor()

    success_count = 0
    for _, row in df.iterrows():
        try:
            # Get bank_id
            package_id = bank_to_package[row['bank']]
            cursor.execute(
                "SELECT bank_id FROM banks WHERE app_package_name = %s",
                (package_id,)
            )
            bank_id = cursor.fetchone()[0]
            
            # Insert review (now with all required columns)
            cursor.execute(
                """
                INSERT INTO reviews (
                    bank_id, review_text, rating, 
                    sentiment_score, sentiment, theme, review_date
                ) VALUES (%s, %s, %s, %s, %s, %s, %s)
                """,
                (
                    bank_id, row['review'], row['rating'],
                    row['sentiment_score'], row['sentiment'],
                    row['theme'], row['date']
                )
            )
            success_count += 1
        except Exception as e:
            print(f"Error inserting {row['bank']}: {str(e)}")

    connection.commit()
    print(f"✅ Successfully inserted {success_count}/{len(df)} records")

except Exception as e:
    print(f"❌ Database error: {e}")
finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()

✅ Successfully inserted 5/5 records


In [11]:
print("Columns in CSV:", df.columns.tolist())

Columns in CSV: ['review_id', 'review', 'rating', 'sentiment_score', 'theme', 'bank', 'date', 'sentiment']
